# Imports y obtención de archivos

In [1]:
import pandas as pd
import math

pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

id2="1DBCzFlJWzdVirjQmoyTaH3GekLTzrznm"
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('googleplaystore_user_reviews.csv')

# Inicialización de los dataframes

*contiene código importante para algunas de las preguntas*

In [ ]:
ps_apps = pd.read_csv("googleplaystore.csv", skiprows=[10473],
                      usecols=["App", "Size", "Category", "Last Updated"])

# Saco las apps duplicadas, me quedo con la primera línea (por ninguna razón en particular)
ps_apps = ps_apps.drop_duplicates(subset=["App"], keep="first")

ps_apps['Last Updated'] = pd.to_datetime(ps_apps['Last Updated'], format="%B %d, %Y")
ps_apps["Category"] = ps_apps["Category"].astype("category")

# Conversión de size a float. Hay valores en mb y kb
def size_to_float(sz):
  if sz == "Varies with device":
    return math.na
  char = sz[-1]
  sz = sz[:-1]
  return float(sz) if char == 'M' else float(sz)/1024
ps_apps["Size"] = ps_apps["Size"].map(size_to_float)

In [ ]:
ps_reviews = pd.read_csv("googleplaystore_user_reviews.csv",
                         usecols=["App", "Sentiment_Polarity", "Sentiment_Subjectivity"])

# Pregunta #35
[*link al notebook general*](https://colab.research.google.com/drive/1MpsnwNHS9Uyfj3PH0kcRIPYK6PhC-bGQ?usp=drive_link)

> *Correlación entre la polaridad y subjetividad promedio de los comentarios de los juegos cuya última actualización haya sido durante el 2018 y cuyo tamaño sea mayor al tamaño promedio de los juegos de ese año. Ignorar las aplicaciones cuyo tamaño varía con el dispositivo (Varies with device).*

**Respuesta:** 0.46

**Notas:**
- Se ignoran los valores de `ps_reviews` con polaridad exactamente igual a 0 porque descubrí con las [visus](https://colab.research.google.com/drive/1Izta0ha_L23-M1VctJ-p5mVxNKUUFYlN) que es altamente probable que la mayoría de los 0.0 hayan sido asignados por error (tldr; la distribución es muy parecida a una normal pero con un pico enorme en 0, y si ignoro los 0 la distribución es más sensible) (For the record, con los 0 la correlatividad es 0.3)
- Como mencioné en la pregunta anterior, el formateo de las fechas y el tamaño se hace durante la etapa de inicialización del df
- Sorry por la cantidad absurda de comentarios en el código pero es que si no me pierdo (?

In [ ]:
#inicializar los df con los datos que necesito
# De las reviews, las columnas de polaridad y subjetividad, agrupadas por app, saco promedio
# Ignoro los valores 0 de polaridad porque están mal, ver las visus
df_right = (ps_reviews[ps_reviews.Sentiment_Polarity != 0]
          [["App", "Sentiment_Polarity", "Sentiment_Subjectivity"]]
          .groupby(["App"])
          .mean())

# De las apps, los que tengan año de last update en 2018 y categoría game,
# Las columnas 'app' y 'size'
df_left = (ps_apps[(ps_apps["Last Updated"].dt.year == 2018) & (ps_apps["Category"] == "GAME")]
            [["App", "Size"]])

# Hago el merge
ps_apps.reset_index()
df_right.reset_index()
df_q35 = pd.merge(
                df_left,
                df_right,
                on="App",
                how="inner")

# Saco tamaño promedio, correlación de polaridad y subjetividad de los que superen el promedio
mean_sz = df_q35["Size"].mean()
df_q35[df_q35["Size"] > mean_sz] \
      [["Sentiment_Polarity", "Sentiment_Subjectivity"]] \
      .corr()

,Sentiment_Polarity,Sentiment_Subjectivity
Sentiment_Polarity,1.00,0.46
Sentiment_Subjectivity,0.46,1.00
